In [15]:
import os
import numpy as np
import librosa
import joblib

# Constants for feature extraction
N_MFCC = 13
N_FFT = 2048
HOP_LENGTH = 512

# Function to convert audio to digital values
def audio_to_digital(audio_file):
    audio, sr = librosa.load(audio_file, sr=16000, res_type='kaiser_fast')
    digital_values = (audio / np.max(np.abs(audio))) * 1023
    return digital_values

# Function to preprocess audio and extract features
def preprocess_audio(file_path, n_mfcc=N_MFCC, n_fft=N_FFT, hop_length=HOP_LENGTH):
    digital_values = audio_to_digital(file_path)
    amplitude = digital_values / 1023

    # Denoising using wavelet
    audio_denoised = denoise_wavelet(amplitude, preserve_threshold=0.05)

    # Extracting features
    mfccs = librosa.feature.mfcc(y=audio_denoised, sr=16000, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    rmse = librosa.feature.rms(y=audio_denoised, frame_length=n_fft, hop_length=hop_length)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio_denoised, sr=16000, n_fft=n_fft, hop_length=hop_length)

    # Calculate mean values
    mean_mfcc = np.mean(mfccs, axis=1)
    mean_rmse = np.mean(rmse)
    mean_centroid = np.mean(spectral_centroid)

    return np.concatenate([mean_mfcc, [mean_rmse], [mean_centroid]]).reshape(1, -1)

# Function to classify audio file
def classify_audio(file_path):
    features = preprocess_audio(file_path)
    prediction = svm_model.predict(features)
    return "Scream" if prediction[0] == 1 else "Conversation"

# Example usage
directory_path = 'contoh'
for file_name in os.listdir(directory_path):
    file_path = os.path.join(directory_path, file_name)
    if os.path.isfile(file_path) and file_name.endswith('.wav'):
        classification = classify_audio(file_path)
        print(f'File: {file_path}, Sound Classification: {classification}')


File: contoh\conversation_851.wav, Sound Classification: Conversation
File: contoh\conversation_861.wav, Sound Classification: Conversation
File: contoh\conversation_866.wav, Sound Classification: Scream
File: contoh\conversation_870.wav, Sound Classification: Scream
File: contoh\conversation_873.wav, Sound Classification: Scream
File: contoh\conversation_877.wav, Sound Classification: Conversation
File: contoh\conversation_880.wav, Sound Classification: Scream
File: contoh\scream_1000.wav, Sound Classification: Scream
File: contoh\scream_1011.wav, Sound Classification: Scream
File: contoh\scream_1024.wav, Sound Classification: Scream
File: contoh\scream_1061.wav, Sound Classification: Scream
File: contoh\scream_1077.wav, Sound Classification: Scream


In [19]:
import os
import numpy as np
import librosa
import joblib
import pywt  # Import library pywt untuk denoising dengan wavelet

# Constants for feature extraction
N_MFCC = 13
N_FFT = 2048
HOP_LENGTH = 512

# Function for denoising using wavelet
def denoise_wavelet(audio, threshold=1e-6, preserve_threshold=0.05):
    coeffs = pywt.wavedec(audio, 'db1', level=6)
    preserved_coeffs = [coeffs[0]]
    
    for i in range(1, len(coeffs)):
        thresholded_coeff = pywt.threshold(coeffs[i], threshold, mode='soft')
        if np.sum(np.abs(thresholded_coeff)) > preserve_threshold:
            preserved_coeffs.append(thresholded_coeff)
        else:
            preserved_coeffs.append(np.zeros_like(thresholded_coeff))

    audio_denoised = pywt.waverec(preserved_coeffs, 'db1')
    return audio_denoised

# Function to convert audio to digital values
def audio_to_digital(audio_file):
    audio, sr = librosa.load(audio_file, sr=16000, res_type='kaiser_fast')
    digital_values = (audio / np.max(np.abs(audio))) * 1023
    return digital_values

# Function to preprocess audio and extract features
def preprocess_audio(file_path, n_mfcc=N_MFCC, n_fft=N_FFT, hop_length=HOP_LENGTH):
    digital_values = audio_to_digital(file_path)
    amplitude = digital_values / 1023

    # Denoising using wavelet
    audio_denoised = denoise_wavelet(amplitude, preserve_threshold=0.05)

    # Extracting features
    mfccs = librosa.feature.mfcc(y=audio_denoised, sr=16000, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    rmse = librosa.feature.rms(y=audio_denoised, frame_length=n_fft, hop_length=hop_length)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio_denoised, sr=16000, n_fft=n_fft, hop_length=hop_length)

    # Calculate mean values
    mean_mfcc = np.mean(mfccs, axis=1)
    mean_rmse = np.mean(rmse)
    mean_centroid = np.mean(spectral_centroid)

    return np.concatenate([mean_mfcc, [mean_rmse], [mean_centroid]]).reshape(1, -1)

# Function to classify audio file
def classify_audio(file_path):
    features = preprocess_audio(file_path)
    prediction = svm_model.predict(features)
    return "Scream" if prediction[0] == 1 else "Conversation"

# Example usage
directory_path = 'percobaan'
for file_name in os.listdir(directory_path):
    file_path = os.path.join(directory_path, file_name)
    if os.path.isfile(file_path) and file_name.endswith('.wav'):
        classification = classify_audio(file_path)
        print(f'File: {file_path}, Sound Classification: {classification}')


File: percobaan\220369__alsarcoli007__scream-05.wav, Sound Classification: Scream
File: percobaan\220651__marionagm90__scream.wav, Sound Classification: Scream
File: percobaan\235590__tcrocker68__girl-scream-short.wav, Sound Classification: Scream
File: percobaan\63639__vtownpunks__screaming-7.wav, Sound Classification: Scream
File: percobaan\chunk_5_2_noise_5.wav, Sound Classification: Conversation
File: percobaan\conversation_851.wav, Sound Classification: Conversation
File: percobaan\conversation_852.wav, Sound Classification: Scream
File: percobaan\conversation_853.wav, Sound Classification: Scream
File: percobaan\conversation_854.wav, Sound Classification: Conversation
File: percobaan\conversation_907.wav, Sound Classification: Scream
File: percobaan\conversation_908.wav, Sound Classification: Conversation
File: percobaan\conversation_909.wav, Sound Classification: Conversation
File: percobaan\conversation_910.wav, Sound Classification: Conversation
File: percobaan\conversation_91

In [4]:
import os
import numpy as np
import librosa
import pywt
import joblib

# Constants for feature extraction
N_MFCC = 13
N_FFT = 2048
HOP_LENGTH = 512

# Load the trained SVM model
model_filename = 'suara_model_svm.pkl'
svm_model = joblib.load(model_filename)

# Function for denoising using wavelet
def denoise_wavelet(audio, threshold=1e-6, preserve_threshold=0.05):
    coeffs = pywt.wavedec(audio, 'db1', level=6)
    preserved_coeffs = [coeffs[0]]
    
    for i in range(1, len(coeffs)):
        thresholded_coeff = pywt.threshold(coeffs[i], threshold, mode='soft')
        if np.sum(np.abs(thresholded_coeff)) > preserve_threshold:
            preserved_coeffs.append(thresholded_coeff)
        else:
            preserved_coeffs.append(np.zeros_like(thresholded_coeff))

    audio_denoised = pywt.waverec(preserved_coeffs, 'db1')
    return audio_denoised

# Function to convert audio to digital values
def audio_to_digital(audio_file):
    audio, sr = librosa.load(audio_file, sr=16000, res_type='kaiser_fast')
    digital_values = (audio / np.max(np.abs(audio))) * 1023
    return digital_values

# Function to preprocess audio and extract features
def preprocess_audio(file_path, n_mfcc=N_MFCC, n_fft=N_FFT, hop_length=HOP_LENGTH):
    digital_values = audio_to_digital(file_path)
    amplitude = digital_values / 1023

    # Denoising using wavelet
    audio_denoised = denoise_wavelet(amplitude, preserve_threshold=0.05)

    # Extracting features
    mfccs = librosa.feature.mfcc(y=audio_denoised, sr=16000, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    rmse = librosa.feature.rms(y=audio_denoised, frame_length=n_fft, hop_length=hop_length)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio_denoised, sr=16000, n_fft=n_fft, hop_length=hop_length)

    # Calculate mean values
    mean_mfcc = np.mean(mfccs, axis=1)
    mean_rmse = np.mean(rmse)
    mean_centroid = np.mean(spectral_centroid)

    return np.concatenate([mean_mfcc, [mean_rmse], [mean_centroid]]).reshape(1, -1)

# Function to classify audio file
def classify_audio(file_path):
    features = preprocess_audio(file_path)
    prediction = svm_model.predict(features)
    return "Scream" if prediction[0] == 1 else "Conversation"

# Example usage
directory_path = 'percobaan'
for file_name in os.listdir(directory_path):
    file_path = os.path.join(directory_path, file_name)
    if os.path.isfile(file_path) and file_name.endswith('.wav'):
        classification = classify_audio(file_path)
        print(f'File: {file_path}, Sound Classification: {classification}')


File: percobaan\conversation_test_1.wav, Sound Classification: Scream
File: percobaan\conversation_test_10.wav, Sound Classification: Scream
File: percobaan\conversation_test_11.wav, Sound Classification: Scream
File: percobaan\conversation_test_2.wav, Sound Classification: Scream
File: percobaan\conversation_test_3.wav, Sound Classification: Scream
File: percobaan\conversation_test_4.wav, Sound Classification: Scream
File: percobaan\conversation_test_5.wav, Sound Classification: Scream
File: percobaan\conversation_test_6.wav, Sound Classification: Scream
File: percobaan\conversation_test_7.wav, Sound Classification: Scream
File: percobaan\conversation_test_8.wav, Sound Classification: Scream
File: percobaan\conversation_test_9.wav, Sound Classification: Scream
File: percobaan\scream_test_1.wav, Sound Classification: Scream
File: percobaan\scream_test_10.wav, Sound Classification: Scream
File: percobaan\scream_test_11.wav, Sound Classification: Scream
File: percobaan\scream_test_12.wav

In [6]:
import os
import numpy as np
import librosa
import pywt
import joblib

# Constants for feature extraction
N_MFCC = 13
N_FFT = 2048
HOP_LENGTH = 512

# Load the trained SVM model
model_filename = 'suara_model_svm.pkl'
svm_model = joblib.load(model_filename)

# Function for denoising using wavelet
def denoise_wavelet(audio, threshold=1e-6, preserve_threshold=0.05):
    coeffs = pywt.wavedec(audio, 'db1', level=6)
    preserved_coeffs = [coeffs[0]]
    
    for i in range(1, len(coeffs)):
        thresholded_coeff = pywt.threshold(coeffs[i], threshold, mode='soft')
        if np.sum(np.abs(thresholded_coeff)) > preserve_threshold:
            preserved_coeffs.append(thresholded_coeff)
        else:
            preserved_coeffs.append(np.zeros_like(thresholded_coeff))

    audio_denoised = pywt.waverec(preserved_coeffs, 'db1')
    return audio_denoised

# Function to convert audio to digital values
def audio_to_digital(audio_file):
    audio, sr = librosa.load(audio_file, sr=16000, res_type='kaiser_fast')
    digital_values = (audio / np.max(np.abs(audio))) * 1023
    return digital_values

# Function to preprocess audio and extract features
def preprocess_audio(file_path, n_mfcc=N_MFCC, n_fft=N_FFT, hop_length=HOP_LENGTH):
    digital_values = audio_to_digital(file_path)
    amplitude = digital_values / 1023

    # Denoising using wavelet
    audio_denoised = denoise_wavelet(amplitude, preserve_threshold=0.05)

    # Extracting features
    mfccs = librosa.feature.mfcc(y=audio_denoised, sr=16000, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    rmse = librosa.feature.rms(y=audio_denoised, frame_length=n_fft, hop_length=hop_length)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio_denoised, sr=16000, n_fft=n_fft, hop_length=hop_length)

    # Calculate mean values
    mean_mfcc = np.mean(mfccs, axis=1)
    mean_rmse = np.mean(rmse)
    mean_centroid = np.mean(spectral_centroid)

    return np.concatenate([mean_mfcc, [mean_rmse], [mean_centroid]]).reshape(1, -1)

# Function to classify audio file
def classify_audio(file_path):
    features = preprocess_audio(file_path)
    prediction = svm_model.predict(features)
    return "Scream" if prediction[0] == 1 else "Conversation"

# Example usage
directory_path = 'percobaan'
for file_name in os.listdir(directory_path):
    file_path = os.path.join(directory_path, file_name)
    if os.path.isfile(file_path) and file_name.endswith('.wav'):
        classification = classify_audio(file_path)
        print(f'File: {file_path}, Sound Classification: {classification}')


File: percobaan\conversation_test_1.wav, Sound Classification: Conversation
File: percobaan\conversation_test_10.wav, Sound Classification: Scream
File: percobaan\conversation_test_11.wav, Sound Classification: Conversation
File: percobaan\conversation_test_2.wav, Sound Classification: Conversation
File: percobaan\conversation_test_3.wav, Sound Classification: Conversation
File: percobaan\conversation_test_4.wav, Sound Classification: Conversation
File: percobaan\conversation_test_5.wav, Sound Classification: Scream
File: percobaan\conversation_test_6.wav, Sound Classification: Conversation
File: percobaan\conversation_test_7.wav, Sound Classification: Scream
File: percobaan\conversation_test_8.wav, Sound Classification: Conversation
File: percobaan\conversation_test_9.wav, Sound Classification: Conversation
File: percobaan\scream_test_1.wav, Sound Classification: Scream
File: percobaan\scream_test_10.wav, Sound Classification: Conversation
File: percobaan\scream_test_11.wav, Sound Cla

In [28]:
import os
import numpy as np
import librosa
import pywt
import joblib

# Constants for feature extraction
N_MFCC = 13
N_FFT = 2048
HOP_LENGTH = 4096

#Terbaik 4096 dan 8192 (salah 4 acc 84%)

# Load the trained SVM model
model_filename = 'suara_model_svm.pkl'
svm_model = joblib.load(model_filename)

# Function for denoising using wavelet
def denoise_wavelet(audio, threshold=1e-6, preserve_threshold=0.05):
    coeffs = pywt.wavedec(audio, 'db1', level=6)
    preserved_coeffs = [coeffs[0]]
    
    for i in range(1, len(coeffs)):
        thresholded_coeff = pywt.threshold(coeffs[i], threshold, mode='soft')
        if np.sum(np.abs(thresholded_coeff)) > preserve_threshold:
            preserved_coeffs.append(thresholded_coeff)
        else:
            preserved_coeffs.append(np.zeros_like(thresholded_coeff))

    audio_denoised = pywt.waverec(preserved_coeffs, 'db1')
    return audio_denoised

# Function to convert audio to digital values
def audio_to_digital(audio_file):
    audio, sr = librosa.load(audio_file, sr=16000, res_type='kaiser_fast')
    digital_values = (audio / np.max(np.abs(audio))) * 1023
    return digital_values

# Function to preprocess audio and extract features
def preprocess_audio(file_path, n_mfcc=N_MFCC, n_fft=N_FFT, hop_length=HOP_LENGTH):
    digital_values = audio_to_digital(file_path)
    amplitude = digital_values / 1023

    # Denoising using wavelet
    audio_denoised = denoise_wavelet(amplitude, preserve_threshold=0.05)

    # Extracting features
    mfccs = librosa.feature.mfcc(y=audio_denoised, sr=16000, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    rmse = librosa.feature.rms(y=audio_denoised, frame_length=n_fft, hop_length=hop_length)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio_denoised, sr=16000, n_fft=n_fft, hop_length=hop_length)

    # Calculate mean values
    mean_mfcc = np.mean(mfccs, axis=1)
    mean_rmse = np.mean(rmse)
    mean_centroid = np.mean(spectral_centroid)

    return np.concatenate([mean_mfcc, [mean_rmse], [mean_centroid]]).reshape(1, -1)

# Function to classify audio file
def classify_audio(file_path):
    features = preprocess_audio(file_path)
    prediction = svm_model.predict(features)
    return "Scream" if prediction[0] == 1 else "Conversation"

# Example usage
directory_path = 'percobaan'
for file_name in os.listdir(directory_path):
    file_path = os.path.join(directory_path, file_name)
    if os.path.isfile(file_path) and file_name.endswith('.wav'):
        classification = classify_audio(file_path)
        print(f'File: {file_path}, Sound Classification: {classification}')


File: percobaan\conversation_test_1.wav, Sound Classification: Conversation
File: percobaan\conversation_test_10.wav, Sound Classification: Scream
File: percobaan\conversation_test_11.wav, Sound Classification: Conversation
File: percobaan\conversation_test_2.wav, Sound Classification: Conversation
File: percobaan\conversation_test_3.wav, Sound Classification: Conversation
File: percobaan\conversation_test_4.wav, Sound Classification: Conversation
File: percobaan\conversation_test_5.wav, Sound Classification: Scream
File: percobaan\conversation_test_6.wav, Sound Classification: Conversation
File: percobaan\conversation_test_7.wav, Sound Classification: Scream
File: percobaan\conversation_test_8.wav, Sound Classification: Conversation
File: percobaan\conversation_test_9.wav, Sound Classification: Conversation
File: percobaan\scream_test_1.wav, Sound Classification: Scream
File: percobaan\scream_test_10.wav, Sound Classification: Scream
File: percobaan\scream_test_11.wav, Sound Classific